# PDFs and consistency

### Author: Chris Wyburn-Powell, [github](https://github.com/chrisrwp/synthetic-ensemble/SIA/SIA_PDFs_consistency.ipynb)

**Input**:
- $\sigma_{mem}$
- $\mu_{mem}$
- $\sigma_{obs}$
- $\mu_{obs}$

**Output**:
- Figure of obs and mem overlapping PDFs with percentiles
- Figure of SIA consistency between models and observations

In [1]:
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.patheffects as pe
import xarray as xr
import datetime

print(datetime.datetime.utcnow().strftime("%H:%M UTC %a %Y-%m-%d"))

16:05 UTC Sun 2021-08-22


## Load data

In [3]:
#sigma and mu mem
sigma_mem = xr.open_dataset(data_path+'SIA/SIA_resampled/Sigma_mem_individual_10000.nc')
mu_mem = xr.open_dataset(data_path+'SIA/SIA_resampled/Mu_mem_individual_10000.nc')

#sigma and mu obs
sigma_obs = xr.open_dataset(data_path+'SIA/SIA_resampled/Sigma_obs_individual_10000.nc')
mu_obs = xr.open_dataset(data_path+'SIA/SIA_resampled/Mu_obs_individual_10000.nc')

In [2]:
month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December']

month_names_short = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

model_names        = ['CanESM2', 'CESM1', 'CSIRO_MK36',  'GFDL_CM3', 'GFDL_ESM2M', 'MPI_ESM1']
model_print_names  = ['CanESM2', 'CESM1', 'CSIRO MK3.6', 'GFDL CM3', 'GFDL ESM2M', 'MPI ESM1']

mem_len   = [50,  40,  30,  20,  30,  100]

colors    = ['m', 'b', 'g', 'orange', 'k', 'tab:olive']

data_path = '/glade/scratch/cwpowell/Synthetic_ensemble/'

# PDFs

In [10]:
def plot_mem_obs_distributions(month_, obs_data, xlims, ylims, letter_start, all_mem=False, means=False, text=False, save_name=False):
    #note means=False should only be used with all_mem=False
    fig, axes = plt.subplots(3,2,figsize=[12,8])
        
    for i, model_name in enumerate(model_names):
        sigma_mem_ = sigma_mem[model_name].sel(month=month_)
        mu_mem_  = mu_mem[model_name].sel(month=month_)
        
        sigma_obs_ = sigma_obs[obs_data].sel(month=month_)
        mu_obs_ = mu_obs[obs_data].sel(month=month_)
        
        ################### plot sigma_mem for all members ##################
        if all_mem:
            for mem_i in range(mem_len[i]):
                #select the correct member's data
                mu    = mu_mem_.isel(member=mem_i)
                sigma = sigma_mem_.isel(member=mem_i)
                #plot fitted line to continous 
                if means:
                    distrib_mems = stats.norm(mu, sigma)
                    x_vals_mems  = np.linspace(mu - 5*sigma, mu + 5*sigma, 100)
                else:
                    distrib_mems = stats.norm(0, sigma)
                    x_vals_mems  = np.linspace(0-5*sigma, 5*sigma, 100)
                probs_mems = [distrib_mems.pdf(x) for x in x_vals_mems]
                axes[s_x[i]][s_y[i]].plot(x_vals_mems, probs_mems, c=colors[i], linewidth=1, alpha=0.35)

        ########################## plot the MEDIAN!! sigma_mem across members ###############################
        mu    = mu_mem_.median('member')
        sigma = sigma_mem_.median('member')

        if means:
            distrib_comp = stats.norm(mu, sigma)
            x_vals_comp  = np.linspace(mu - 5*sigma, mu + 5*sigma, 100)
        else:
            distrib_comp = stats.norm(0, sigma)
            x_vals_comp  = np.linspace(0 - 5*sigma, 5*sigma, 100)
        probs_comp = [distrib_comp.pdf(x) for x in x_vals_comp]
        axes[s_x[i]][s_y[i]].plot(x_vals_comp, probs_comp, c=colors[i], linewidth=3, alpha=1,
                                  path_effects=[pe.Stroke(linewidth=6, foreground='1'), pe.Normal()])

        ############################ plot sigma_obs #################################
        if means:
            distrib_obs = stats.norm(mu_obs_, sigma_obs_)
            x_vals_obs  = np.linspace(mu_obs_ - 5*sigma_obs_, 
                                      mu_obs_ + 5*sigma_obs_, 100)
        else:
            distrib_obs = stats.norm(0, sigma_obs_)
            x_vals_obs  = np.linspace(0 - 5*sigma_obs_, 5*sigma_obs_, 100)
        probs_obs = [distrib_obs.pdf(x) for x in x_vals_obs]
        axes[s_x[i]][s_y[i]].plot(x_vals_obs, probs_obs, c='r', linestyle='--', linewidth=2)

        ########################### add text and title to subplot ##################################
        if text:
            element = np.nanargmin(abs(np.sort(mu_mem_-mu_obs_)))
            if element == 0:
                percentile_mu = '0'
            else:
                percentile_mu = str(int(((element+1)/mem_len[i])*100))

            axes[s_x[i]][s_y[i]].annotate('Percentile '+r'${} ={}\%$'.format('\mu_{obs}',percentile_mu),
                                          xy=(0.58, 0.89), xycoords='axes fraction', fontsize=12, color=colors[i]);


            element = np.nanargmin(abs(np.sort(sigma_mem_-sigma_obs_)))
            if element == 0:
                percentile_mem = '0'
            else:
                percentile_mem = str(int(((element+1)/mem_len[i])*100))

            axes[s_x[i]][s_y[i]].annotate('Percentile '+r'${} ={}\%$'.format('\sigma_{obs}',percentile_mem),
                                          xy=(0.03, 0.89), xycoords='axes fraction', fontsize=12, color=colors[i]);

        #add title
        axes[s_x[i]][s_y[i]].set_title('({}) {}'.format(letter_list[i+letter_start], model_print_names[i]))
        #add axes titles and labels
        axes[s_x[i]][s_y[i]].set_xlim(xlims[0], xlims[1])
        axes[s_x[i]][s_y[i]].set_ylim(ylims[0], ylims[1])

        if i%2 == 0:
            axes[s_x[i]][s_y[i]].set_ylabel('Normalized Frequency', fontsize=12)

        if xlab_bool[i]: axes[s_x[i]][s_y[i]].set_xlabel(r'$Standard \ Deviation \ of \ SIA \ [10^6 \ km^2]$', fontsize=12)

    
    plt.suptitle(month_names[month_-1], fontsize=16, fontweight='bold')
    
    legend_elements = [Line2D([0], [0], color='r', ls='--', lw=3, label='Observations'),
                       Line2D([0], [0], color=colors[0], lw=3, label='CanESM2'),
                       Line2D([0], [0], color=colors[1], lw=3, label='CESM1'),
                       Line2D([0], [0], color=colors[2], lw=3, label='CSIRO MK3.6'),
                       Line2D([0], [0], color=colors[3], lw=3, label='GFDL CM3'),
                       Line2D([0], [0], color=colors[4], lw=3, label='GFDL ESM2M'),
                       Line2D([0], [0], color=colors[5], lw=3, label='MPI ESM1')]

    extra_legend = plt.legend(handles=legend_elements, bbox_to_anchor=(-0.1, -0.5), loc='lower center', borderaxespad=0., ncol=7)

    plt.gca().add_artist(extra_legend)
    
    plt.subplots_adjust(top = 0.93, hspace = 0.28, wspace = 0.09) #best to use 0.3
    
    #save figure
    if save_name:            
        plt.tight_layout()
        fig.savefig(data_path+'SIA/figures/SIA_PDFs_consistency/PDFs_mem_obs_{}_{}.pdf'.format(obs_data, str(month_).zfill(2)))
        fig.savefig(data_path+'SIA/figures/SIA_PDFs_consistency/PDFs_mem_obs_{}_{}.png'.format(obs_data, str(month_).zfill(2)))

In [20]:
s_y = [0,1,0,1,0,1]
s_x = [0,0,1,1,2,2]

letter_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

             #Jan  Feb   Mar   Apr  May   Jun   Jul  Aug  Sep Oct   Nov   Dec
max_x_lim = [0.55, 0.55, 0.66, 0.7, 0.55, 0.55, 0.7, 0.7, 0.85, 1.1,  1, 0.55]
max_y_lim = [  27,   22,  30,   20,   26,   30,  27,  29,   33,  31, 27,   27]

xlab_bool   = [False, False, False, False, True, True]

#135 max for composite CLIVAR PDF is fine, x limits of -0.03 and 0.03 
i = 9
plot_mem_obs_distributions(i, 'HadISST1', [0.12, max_x_lim[i-1]], [0, max_y_lim[i-1]], 0, all_mem=True, means=True, text=True, save_name=True)

# Consistency

In [19]:
#define calculation of consistency between sigma and mu mem and obs
def consistent_threshold(sigma_mem_stat, sigma_obs_stat, threshold):

    ones_twelve_five = xr.DataArray(data   = np.ones([12,6]),
                                    coords = {'month':np.arange(1,13,1), 'model':model_names},
                                    dims   = {'month':np.arange(1,13,1), 'model':model_names})
    
    #find the difference of the minimum and maximum values from observations 
    mins = sigma_mem_stat.min('member') - sigma_obs_stat*(1+threshold)
    maxs = sigma_mem_stat.max('member') - sigma_obs_stat*(1-threshold)

    #find where min is below observations AND max is above observations, fill 1 for consistent, 0 for not
    consistent = mins.where(mins <= 0) * maxs.where(maxs >= 0)
    consistent = xr.ufuncs.logical_not(xr.ufuncs.isnan(consistent)) #returns True for consistent, False for inconsistent
    consistent = ones_twelve_five.copy().where(consistent==True, 0) #now 1 if consistent, 0 if not

    too_high = ones_twelve_five.copy().where(xr.ufuncs.logical_not(xr.ufuncs.isnan(mins.where(mins > 0))),0) 
    too_low  = ones_twelve_five.copy().where(xr.ufuncs.logical_not(xr.ufuncs.isnan(maxs.where(maxs < 0))),0)
    
    return(consistent, too_high, too_low)

In [22]:
#make data with model as a dimension
sigma_mem_cons = sigma_mem.to_array(dim='model')
mu_mem_cons    = mu_mem.to_array(dim='model')

for obs_data in list(sigma_obs.keys()):

    #select the dataset for the observations
    sigma_obs_cons = sigma_obs[obs_data]
    mu_obs_cons    = mu_obs[obs_data]
    
    #calculate which months and models are consistent 
    sigmas = consistent_threshold(sigma_mem_cons, sigma_obs_cons, 0) 
    mus    = consistent_threshold(mu_mem_cons, mu_obs_cons, 0)
    
    #determine red, blue or white with +1, -1, 0
    sigmas_net = sigmas[1] + -1*sigmas[2] 
    mus_net    = mus[1] + -1*mus[2]

    both = sigmas_net + mus_net #data for panel (c)
    all_data = [sigmas_net, mus_net, both] #data for (a), (b) and (c) together
    
    titles = [r'$(a) \ Consistent \ for \ \sigma$', r'$(b) \ Consistent \ for \ \mu$', 
              r'$(c) \ Consistent \ for \ \sigma \ and \ \mu$']
    cmap = mpl.colors.LinearSegmentedColormap.from_list("", ["red","1","blue"])
    
    #do the plotting
    fig, ax = plt.subplots(1,3,figsize=[10,6]) 

    for i in range(3):
        ax[i].pcolor(all_data[i].reindex(month=list(reversed(all_data[i].month))), cmap=cmap, edgecolor='black', lw=2)
        ax[i].set_xticks(np.arange(0.5, 5.6, 1));
        ax[i].set_yticks(np.arange(0.5, 11.6, 1));
        ax[i].set_xticklabels(model_print_names, fontsize=14,rotation = 90);
        ax[i].set_yticklabels(month_names_short.copy()[::-1], fontsize=14);
        ax[i].set_title(titles[i], fontdict={'size':15, 'weight':'bold'});

    plt.tight_layout();
    plt.savefig(data_path+'SIA/figures/SIA_PDFs_consistency/Consistent_mem_obs_10000_{}.pdf'.format(obs_data))
    plt.savefig(data_path+'SIA/figures/SIA_PDFs_consistency/Consistent_mem_obs_10000_{}.png'.format(obs_data))